In [1]:
import os
import sys; sys.path.append("~/HHbbgg_ETH/Training/python") # to load packages
import training_utils as utils
import numpy as np
reload(utils)

Welcome to ROOTaaS 6.05/02


<module 'training_utils' from '/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/Training/python/training_utils.pyc'>

In [2]:
reload(utils)
ntuples = '20170620'
# "%" sign allows to interpret the rest as a system command
%env data=$utils.IO.ldata$ntuples
files = ! ls $data | sort -t_ -k 3 -n
#pick only SM
signal = [s for s in files if "GluGluToHHTo2B2G_node_SM" in s]
diphotonJets = [s for s in files if "DiPhotonJets" in s]
gJets_lowPt = [s for s in files if "GJet_Pt-20to40" in s]
gJets_highPt = [s for s in files if "GJet_Pt-40" in s]
ggH = [s for s in files if "GluGluHToGG" in s]
vbf = [s for s in files if "VBFHToGG" in s]
VH = [s for s in files if "VHToGG" in s]
bbH = [s for s in files if "bbHToGG_M-125_4FS_yb2" in s] #is this correct?
ttH = [s for s in files if "ttHToGG" in s]

Data= [s for s in files if "DoubleEG" in s]

utils.IO.add_signal(ntuples,signal,1)
utils.IO.add_background(ntuples,diphotonJets,-1)
utils.IO.add_background(ntuples,gJets_lowPt,-2)
utils.IO.add_background(ntuples,gJets_highPt,-2)
utils.IO.add_background(ntuples,ggH,-3)
utils.IO.add_background(ntuples,vbf,-4)
utils.IO.add_background(ntuples,VH,-5)
utils.IO.add_background(ntuples,bbH,-6)
utils.IO.add_background(ntuples,ttH,-7)

utils.IO.add_data(ntuples,Data,-10)


for i in range(len(utils.IO.backgroundName)):        
    print "using background file n."+str(i)+": "+utils.IO.backgroundName[i]
for i in range(len(utils.IO.signalName)):    
    print "using signal file n."+str(i)+": "+utils.IO.signalName[i]
print "using data file: "+ utils.IO.dataName[0]

env: data=/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620
using background file n.0: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_DiPhotonJetsBox_MGG-80toInf_13TeV-Sherpa.root
using background file n.1: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_GJet_Pt-20to40_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8.root
using background file n.2: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_GJet_Pt-40toInf_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8.root
using background file n.3: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_GluGluHToGG_M-125_13TeV_powheg_pythia8.root
using background file n.4: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_VBFHToGG_M-125_13TeV_powheg_pythia8.root
using background file n.5: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_VHToGG_M125_13TeV_amcatnloFXFX_madspin_pythia8.

In [3]:
#use noexpand for root expressions, it needs this file https://github.com/ibab/root_pandas/blob/master/root_pandas/readwrite.py
#all variables
branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg),noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M(),customLeadingPhotonIDMVA,customSubLeadingPhotonIDMVA,leadingPhotonSigOverE,subleadingPhotonSigOverE,sigmaMOverMDecorr,DiJetDiPho_DR'.split(",")
#std variables
#branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg),noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M()'.split(",")
branch_names = [c.strip() for c in branch_names]
print branch_names

import pandas as pd
import root_pandas as rpd

# no need to shuffle here, we just count events
utils.preprocessing.set_signals_and_backgrounds("bbggSelectionTree",branch_names,shuffle=False)
X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig=utils.preprocessing.set_variables(branch_names)

X_data,y_data,weights_data = utils.preprocessing.set_data("bbggSelectionTree",branch_names)
X_data,y_data,weights_data = utils.preprocessing.clean_signal_events_single_dataset(X_data,y_data,weights_data)

#bbggTrees have by default signal and CR events, let's be sure that we clean it
X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig=utils.preprocessing.clean_signal_events(X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig)

#np.savetxt('/tmp/micheli/xbkg.txt',X_bkg)
#utils.preprocessing.adjust_and_compress(utils.IO.background_df[0]).to_csv('/tmp/micheli/bkg.csv')


['leadingJet_bDis', 'subleadingJet_bDis', 'noexpand:fabs(CosThetaStar_CS)', 'noexpand:fabs(CosTheta_bb)', 'noexpand:fabs(CosTheta_gg)', 'noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M()', 'noexpand:dijetCandidate.Pt()/diHiggsCandidate.M()', 'customLeadingPhotonIDMVA', 'customSubLeadingPhotonIDMVA', 'leadingPhotonSigOverE', 'subleadingPhotonSigOverE', 'sigmaMOverMDecorr', 'DiJetDiPho_DR']


In [4]:
# load the model from disk
from sklearn.externals import joblib
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH/Training/output_files/allMC_stdVariables.pkl'))
loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH/Training/output_files/allMC_resWeighting_F.pkl'))

bkg_0 = X_bkg[y_bkg ==-1]
bkg_1 = X_bkg[y_bkg ==-2]
bkg_3 = X_bkg[y_bkg ==-3]
bkg_4 = X_bkg[y_bkg ==-4]
bkg_5 = X_bkg[y_bkg ==-5]
bkg_6 = X_bkg[y_bkg ==-6]
bkg_7 = X_bkg[y_bkg ==-7]



#bkg_1 = np.concatenate((X_total_train[y_total_train == -2],X_total_test[y_total_test == -2]))
#bkg_3 = np.concatenate((X_total_train[y_total_train == -3],X_total_test[y_total_test == -3]))
#bkg_4 = np.concatenate((X_total_train[y_total_train == -4],X_total_test[y_total_test == -4]))
#bkg_5 = np.concatenate((X_total_train[y_total_train == -5],X_total_test[y_total_test == -5]))
#bkg_6 = np.concatenate((X_total_train[y_total_train == -6],X_total_test[y_total_test == -6]))
#bkg_7 = np.concatenate((X_total_train[y_total_train == -7],X_total_test[y_total_test == -7]))


Y_pred_sig = loaded_model.predict_proba(X_sig)[:,2].astype(np.float64)
Y_pred_bkg_0 = loaded_model.predict_proba(bkg_0)[:,2].astype(np.float64)
Y_pred_bkg_1 = loaded_model.predict_proba(bkg_1)[:,2].astype(np.float64)
Y_pred_bkg_3 = loaded_model.predict_proba(bkg_3)[:,2].astype(np.float64)
Y_pred_bkg_4 = loaded_model.predict_proba(bkg_4)[:,2].astype(np.float64)
Y_pred_bkg_5 = loaded_model.predict_proba(bkg_5)[:,2].astype(np.float64)
Y_pred_bkg_6 = loaded_model.predict_proba(bkg_6)[:,2].astype(np.float64)
Y_pred_bkg_7 = loaded_model.predict_proba(bkg_7)[:,2].astype(np.float64)

Y_pred_data = loaded_model.predict_proba(X_data)[:,2].astype(np.float64)

print Y_pred_data 



[ 0.00224534  0.20553288  0.00474526 ...,  0.03072882  0.05181648
  0.6292057 ]


In [5]:
#define MVA cut and additional variables needed
MVA_Cut = 0.92
additionalCut_names = 'noexpand:diphotonCandidate.M(),noexpand:dijetCandidate.M(),MX,isSignal'.split(",")
outTag = '20170711_highMassCut'


In [11]:
#### here counting events only in dijet mass window######
#print utils.IO.signal_df[0]['proc']
#print utils.IO.signal_df[0].index
sig_count_df = rpd.read_root(utils.IO.signalName[0],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(sig_count_df,utils.IO.sigProc[0],utils.IO.signalName[0])

#nTot is a multidim vector with all additional variables, dictVar is a dictionary associating a name of the variable
#to a position in the vector
nTot,dictVar = utils.postprocessing.stackFeatures(sig_count_df,additionalCut_names)
print nTot.shape
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,dictVar['weight']]!=0),:][0]
print nCleaned.shape
#apply MVA cut at 90% eff
print "Y_pred"
print Y_pred_sig.shape

#it's a stupid way to save also mva output after cuts, more efficient solutions should be found
Y_pred_total = np.column_stack((nCleaned,Y_pred_sig))

nCleaned_MVA = nCleaned[np.where(Y_pred_sig>MVA_Cut)]
#apply massWindow +- 1.5
print nCleaned_MVA.shape
nCleaned_massWindow = utils.postprocessing.cutInvariantMass(nCleaned_MVA,dictVar['diphotonCandidate.M()'],123.5000,126.5000)
nEvts = sum(nCleaned_massWindow[:,dictVar['weight']])
print "signal events with 36 fb-1: "+str(nEvts)
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet = utils.postprocessing.cutInvariantMass(nCleaned_massWindow,dictVar['dijetCandidate.M()'],100.,140.)
nEvts_Cut = sum(nCleaned_mdijet[:,dictVar['weight']])
print "after cutting in dijet mass:"+str(nEvts_Cut)

processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.signalName[0].split("/")[len(utils.IO.signalName[0].split("/"))-1].replace("output_","")
utils.postprocessing.saveTree(processPath,dictVar,nCleaned_mdijet)

processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+"SigWS_highMass.root"
nCleaned_MVA_2 = Y_pred_total[np.where(Y_pred_sig>MVA_Cut)]
#

nCleaned_massWindow_2 = utils.postprocessing.cutInvariantMass(nCleaned_MVA_2,dictVar['diphotonCandidate.M()'],100,180)

nCleaned_mdijet_2 = utils.postprocessing.cutInvariantMass(nCleaned_massWindow_2,dictVar['dijetCandidate.M()'],100.,140.)
nCleaned_MX = utils.postprocessing.applyCut(nCleaned_mdijet_2,dictVar['MX'],350)


Y_pred_cut=nCleaned_MX[:,nCleaned_MX.shape[1]-1].astype(np.float64)


utils.postprocessing.saveTree(processPath,dictVar,nCleaned_MX,Y_pred_cut,nameTree="reducedTree_sig")


processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.signalName[0].split("/")[len(utils.IO.signalName[0].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
utils.postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_sig)

(21184, 5)
(18646, 5)
Y_pred
(18646,)
(12331, 5)
signal events with 36 fb-1: 7.19878563033
after cutting in dijet mass:4.86803561373


In [191]:
bkg_0_count_df = rpd.read_root(utils.IO.backgroundName[0],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_0_count_df,utils.IO.bkgProc[0],utils.IO.backgroundName[0])

nTot,dictVar = utils.postprocessing.stackFeatures(bkg_0_count_df,additionalCut_names)

print nTot.shape
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,dictVar['weight']]!=0),:][0]
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_0>MVA_Cut)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindow = utils.postprocessing.cutInvariantMass(nCleaned_MVA,dictVar['diphotonCandidate.M()'],123.5000,126.5000)
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,dictVar['weight']])
print utils.IO.backgroundName[0].split("/")[len(utils.IO.backgroundName[0].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
print "with scale factor:"+str(nEvts*1.45) 
#Histo_B = np.histogram(nCleaned_massWindow[:,2],bins=30,range=(100,180),weights=nCleaned_massWindow[:,1])
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet = utils.postprocessing.cutInvariantMass(nCleaned_massWindow,dictVar['dijetCandidate.M()'],100.,140.)
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,dictVar['weight']])
print "after cutting in dijet mass (with scale factor):"+str(nEvts_Cut*1.45)

crazySF=1.45


processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[0].split("/")[len(utils.IO.backgroundName[0].split("/"))-1].replace("output_","")
utils.postprocessing.saveTree(processPath,dictVar,nCleaned_mdijet,SF=crazySF)

processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[0].split("/")[len(utils.IO.backgroundName[0].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
utils.postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_bkg_0,SF=crazySF)


(2190915, 5)
(1900684, 5)
(26544, 5)
(1263, 5)
DiPhotonJetsBox_MGG-80toInf_13TeV-Sherpa events with 36 fb-1: 17.4511303435
with scale factor:25.3041389981
(383, 5)
after cutting in dijet mass (with scale factor):7.49152865668


In [192]:
bkg_1_count_df = rpd.read_root(utils.IO.backgroundName[1],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_1_count_df,utils.IO.bkgProc[1],utils.IO.backgroundName[1])

crazySF_20=25
nTot,dictVar = utils.postprocessing.stackFeatures(bkg_1_count_df,additionalCut_names,SF=crazySF_20)

print nTot.shape

bkg_2_count_df = rpd.read_root(utils.IO.backgroundName[2],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_2_count_df,utils.IO.bkgProc[2],utils.IO.backgroundName[2])

crazySF_40=3
nTot_2,dictVar = utils.postprocessing.stackFeatures(bkg_2_count_df,additionalCut_names,SF=crazySF_40)


nTot_3 = np.concatenate((nTot,nTot_2))

print nTot_3.shape
#apply isSignal cleaning
nCleaned = nTot_3[np.where(nTot_3[:,dictVar['weight']]!=0),:][0]
print "nCleaned"
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_1>MVA_Cut)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindow = utils.postprocessing.cutInvariantMass(nCleaned_MVA,dictVar['diphotonCandidate.M()'],123.5000,126.5000)
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,dictVar['weight']])
print utils.IO.backgroundName[2].split("/")[len(utils.IO.backgroundName[2].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
nCleaned_mdijet = utils.postprocessing.cutInvariantMass(nCleaned_massWindow,dictVar['dijetCandidate.M()'],100.,140.)
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,dictVar['weight']])
print "after cutting in dijet mass:"+str(nEvts_Cut)

#np.savetxt('/tmp/micheli/prova7.txt',nCleaned_mdijet)
#utils.preprocessing.adjust_and_compress(bkg_2_count_df).to_csv('/tmp/micheli/background_7_2.csv') 
processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[1].split("/")[len(utils.IO.backgroundName[1].split("/"))-1].replace("output_","").replace("_Pt-20to40","")
utils.postprocessing.saveTree(processPath,dictVar,nCleaned_mdijet)


processPath=(os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[1].split("/")[len(utils.IO.backgroundName[1].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root").replace("_Pt-20to40","")
utils.postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_bkg_1)




(2590, 5)
(63259, 5)
nCleaned
(22278, 5)
(20, 5)
(0, 5)
GJet_Pt-40toInf_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8 events with 36 fb-1: 0
(0, 5)
after cutting in dijet mass:0


In [12]:
bkg_3_count_df = rpd.read_root(utils.IO.backgroundName[3],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_3_count_df,utils.IO.bkgProc[3],utils.IO.backgroundName[3])

nTot,dictVar = utils.postprocessing.stackFeatures(bkg_3_count_df,additionalCut_names)


nCleaned = nTot[np.where(nTot[:,dictVar['weight']]!=0),:][0]
print "nCleaned"
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_3>MVA_Cut)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindow = utils.postprocessing.cutInvariantMass(nCleaned_MVA,dictVar['diphotonCandidate.M()'],123.5000,126.5000)
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,dictVar['weight']])
print utils.IO.backgroundName[3].split("/")[len(utils.IO.backgroundName[3].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
nCleaned_mdijet = utils.postprocessing.cutInvariantMass(nCleaned_massWindow,dictVar['dijetCandidate.M()'],100.,140.)
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,dictVar['weight']])
print "after cutting in dijet mass:"+str(nEvts_Cut)


processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[3].split("/")[len(utils.IO.backgroundName[3].split("/"))-1].replace("output_","")
utils.postprocessing.saveTree(processPath,dictVar,nCleaned_mdijet)


processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[3].split("/")[len(utils.IO.backgroundName[3].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
utils.postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_bkg_3)


processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[3].split("/")[len(utils.IO.backgroundName[3].split("/"))-1].replace("output_","").replace(".root","")+"WS_highMass.root"
#it's a stupid way to save also mva output after cuts, more efficient solutions should be found
Y_pred_total = np.column_stack((nCleaned,Y_pred_bkg_3))

nCleaned_MVA_2 = Y_pred_total[np.where(Y_pred_bkg_3>MVA_Cut)]
#

nCleaned_massWindow_2 = utils.postprocessing.cutInvariantMass(nCleaned_MVA_2,dictVar['diphotonCandidate.M()'],100,180)

nCleaned_mdijet_2 = utils.postprocessing.cutInvariantMass(nCleaned_massWindow_2,dictVar['dijetCandidate.M()'],100.,140.)
nCleaned_MX = utils.postprocessing.applyCut(nCleaned_mdijet_2,dictVar['MX'],350)


Y_pred_cut=nCleaned_MX[:,nCleaned_MX.shape[1]-1].astype(np.float64)


utils.postprocessing.saveTree(processPath,dictVar,nCleaned_MX,Y_pred_cut,nameTree="reducedTree_bkg_3")

nCleaned
(56382, 5)
(1514, 5)
(1310, 5)
GluGluHToGG_M-125_13TeV_powheg_pythia8 events with 36 fb-1: 4.63500336751
(389, 5)
after cutting in dijet mass:1.33747775696


In [13]:
bkg_4_count_df = rpd.read_root(utils.IO.backgroundName[4],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_4_count_df,utils.IO.bkgProc[4],utils.IO.backgroundName[4])

nTot,dictVar = utils.postprocessing.stackFeatures(bkg_4_count_df,additionalCut_names)


nCleaned = nTot[np.where(nTot[:,dictVar['weight']]!=0),:][0]
print "nCleaned"
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_4>MVA_Cut)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindow = utils.postprocessing.cutInvariantMass(nCleaned_MVA,dictVar['diphotonCandidate.M()'],123.5000,126.5000)
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,dictVar['weight']])
print utils.IO.backgroundName[4].split("/")[len(utils.IO.backgroundName[4].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
nCleaned_mdijet = utils.postprocessing.cutInvariantMass(nCleaned_massWindow,dictVar['dijetCandidate.M()'],100.,140.)
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,dictVar['weight']])
print "after cutting in dijet mass:"+str(nEvts_Cut)


processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[4].split("/")[len(utils.IO.backgroundName[4].split("/"))-1].replace("output_","")
utils.postprocessing.saveTree(processPath,dictVar,nCleaned_mdijet)


processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[4].split("/")[len(utils.IO.backgroundName[4].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
utils.postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_bkg_4)

processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[4].split("/")[len(utils.IO.backgroundName[4].split("/"))-1].replace("output_","").replace(".root","")+"WS_highMass.root"
#it's a stupid way to save also mva output after cuts, more efficient solutions should be found
Y_pred_total = np.column_stack((nCleaned,Y_pred_bkg_4))

nCleaned_MVA_2 = Y_pred_total[np.where(Y_pred_bkg_4>MVA_Cut)]
#

nCleaned_massWindow_2 = utils.postprocessing.cutInvariantMass(nCleaned_MVA_2,dictVar['diphotonCandidate.M()'],100,180)

nCleaned_mdijet_2 = utils.postprocessing.cutInvariantMass(nCleaned_massWindow_2,dictVar['dijetCandidate.M()'],100.,140.)
nCleaned_MX = utils.postprocessing.applyCut(nCleaned_mdijet_2,dictVar['MX'],350)


Y_pred_cut=nCleaned_MX[:,nCleaned_MX.shape[1]-1].astype(np.float64)


utils.postprocessing.saveTree(processPath,dictVar,nCleaned_MX,Y_pred_cut,nameTree="reducedTree_bkg_4")

nCleaned
(76431, 5)
(2253, 5)
(1923, 5)
VBFHToGG_M-125_13TeV_powheg_pythia8 events with 36 fb-1: 0.582927454773
(565, 5)
after cutting in dijet mass:0.173842158377


In [14]:
bkg_5_count_df = rpd.read_root(utils.IO.backgroundName[5],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_5_count_df,utils.IO.bkgProc[5],utils.IO.backgroundName[5])

nTot,dictVar = utils.postprocessing.stackFeatures(bkg_5_count_df,additionalCut_names)


nCleaned = nTot[np.where(nTot[:,dictVar['weight']]!=0),:][0]
print "nCleaned"
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_5>MVA_Cut)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindow = utils.postprocessing.cutInvariantMass(nCleaned_MVA,dictVar['diphotonCandidate.M()'],123.5000,126.5000)
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,dictVar['weight']])
print utils.IO.backgroundName[5].split("/")[len(utils.IO.backgroundName[5].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
nCleaned_mdijet = utils.postprocessing.cutInvariantMass(nCleaned_massWindow,dictVar['dijetCandidate.M()'],100.,140.)
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,dictVar['weight']])
print "after cutting in dijet mass:"+str(nEvts_Cut)


processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[5].split("/")[len(utils.IO.backgroundName[5].split("/"))-1].replace("output_","")
utils.postprocessing.saveTree(processPath,dictVar,nCleaned_mdijet)


processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[5].split("/")[len(utils.IO.backgroundName[5].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
utils.postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_bkg_5)


processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[5].split("/")[len(utils.IO.backgroundName[5].split("/"))-1].replace("output_","").replace(".root","")+"WS_highMass.root"
#it's a stupid way to save also mva output after cuts, more efficient solutions should be found
Y_pred_total = np.column_stack((nCleaned,Y_pred_bkg_5))

nCleaned_MVA_2 = Y_pred_total[np.where(Y_pred_bkg_5>MVA_Cut)]
#

nCleaned_massWindow_2 = utils.postprocessing.cutInvariantMass(nCleaned_MVA_2,dictVar['diphotonCandidate.M()'],100,180)

nCleaned_mdijet_2 = utils.postprocessing.cutInvariantMass(nCleaned_massWindow_2,dictVar['dijetCandidate.M()'],100.,140.)
nCleaned_MX = utils.postprocessing.applyCut(nCleaned_mdijet_2,dictVar['MX'],350)


Y_pred_cut=nCleaned_MX[:,nCleaned_MX.shape[1]-1].astype(np.float64)


utils.postprocessing.saveTree(processPath,dictVar,nCleaned_MX,Y_pred_cut,nameTree="reducedTree_bkg_5")

nCleaned
(57455, 5)
(3989, 5)
(3259, 5)
VHToGG_M125_13TeV_amcatnloFXFX_madspin_pythia8 events with 36 fb-1: 1.79602269549
(857, 5)
after cutting in dijet mass:0.420635549789


In [15]:
bkg_6_count_df = rpd.read_root(utils.IO.backgroundName[6],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_6_count_df,utils.IO.bkgProc[6],utils.IO.backgroundName[6])

nTot,dictVar = utils.postprocessing.stackFeatures(bkg_6_count_df,additionalCut_names)


nCleaned = nTot[np.where(nTot[:,dictVar['weight']]!=0),:][0]
print "nCleaned"
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_6>MVA_Cut)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindow = utils.postprocessing.cutInvariantMass(nCleaned_MVA,dictVar['diphotonCandidate.M()'],123.5000,126.5000)
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,dictVar['weight']])
print utils.IO.backgroundName[6].split("/")[len(utils.IO.backgroundName[1].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
nCleaned_mdijet = utils.postprocessing.cutInvariantMass(nCleaned_massWindow,dictVar['dijetCandidate.M()'],100.,140.)
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,dictVar['weight']])
print "after cutting in dijet mass:"+str(nEvts_Cut)


processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[6].split("/")[len(utils.IO.backgroundName[6].split("/"))-1].replace("output_","")
utils.postprocessing.saveTree(processPath,dictVar,nCleaned_mdijet)


processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[6].split("/")[len(utils.IO.backgroundName[6].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
utils.postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_bkg_6)

processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[6].split("/")[len(utils.IO.backgroundName[6].split("/"))-1].replace("output_","").replace(".root","")+"WS_highMass.root"
#it's a stupid way to save also mva output after cuts, more efficient solutions should be found
Y_pred_total = np.column_stack((nCleaned,Y_pred_bkg_6))

nCleaned_MVA_2 = Y_pred_total[np.where(Y_pred_bkg_6>MVA_Cut)]
#

nCleaned_massWindow_2 = utils.postprocessing.cutInvariantMass(nCleaned_MVA_2,dictVar['diphotonCandidate.M()'],100,180)

nCleaned_mdijet_2 = utils.postprocessing.cutInvariantMass(nCleaned_massWindow_2,dictVar['dijetCandidate.M()'],100.,140.)
nCleaned_MX = utils.postprocessing.applyCut(nCleaned_mdijet_2,dictVar['MX'],350)


Y_pred_cut=nCleaned_MX[:,nCleaned_MX.shape[1]-1].astype(np.float64)


utils.postprocessing.saveTree(processPath,dictVar,nCleaned_MX,Y_pred_cut,nameTree="reducedTree_bkg_6")

nCleaned
(106395, 5)
(10678, 5)
(8750, 5)
bbHToGG_M-125_4FS_yb2_13TeV_amcatnlo events with 36 fb-1: 0.492642179582
(3130, 5)
after cutting in dijet mass:0.184155315158


In [16]:
bkg_7_count_df = rpd.read_root(utils.IO.backgroundName[7],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_7_count_df,utils.IO.bkgProc[7],utils.IO.backgroundName[7])

nTot,dictVar = utils.postprocessing.stackFeatures(bkg_7_count_df,additionalCut_names)


nCleaned = nTot[np.where(nTot[:,dictVar['weight']]!=0),:][0]
print "nCleaned"
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_7>MVA_Cut)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindow = utils.postprocessing.cutInvariantMass(nCleaned_MVA,dictVar['diphotonCandidate.M()'],123.5000,126.5000)
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,dictVar['weight']])
print utils.IO.backgroundName[7].split("/")[len(utils.IO.backgroundName[1].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
nCleaned_mdijet = utils.postprocessing.cutInvariantMass(nCleaned_massWindow,dictVar['dijetCandidate.M()'],100.,140.)
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,dictVar['weight']])
print "after cutting in dijet mass:"+str(nEvts_Cut)


processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[7].split("/")[len(utils.IO.backgroundName[7].split("/"))-1].replace("output_","")
utils.postprocessing.saveTree(processPath,dictVar,nCleaned_mdijet)


processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[7].split("/")[len(utils.IO.backgroundName[7].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
utils.postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_bkg_7)

processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[7].split("/")[len(utils.IO.backgroundName[7].split("/"))-1].replace("output_","").replace(".root","")+"WS_highMass.root"
#it's a stupid way to save also mva output after cuts, more efficient solutions should be found
Y_pred_total = np.column_stack((nCleaned,Y_pred_bkg_7))

nCleaned_MVA_2 = Y_pred_total[np.where(Y_pred_bkg_7>MVA_Cut)]
#

nCleaned_massWindow_2 = utils.postprocessing.cutInvariantMass(nCleaned_MVA_2,dictVar['diphotonCandidate.M()'],100,180)

nCleaned_mdijet_2 = utils.postprocessing.cutInvariantMass(nCleaned_massWindow_2,dictVar['dijetCandidate.M()'],100.,140.)
nCleaned_MX = utils.postprocessing.applyCut(nCleaned_mdijet_2,dictVar['MX'],350)


Y_pred_cut=nCleaned_MX[:,nCleaned_MX.shape[1]-1].astype(np.float64)


utils.postprocessing.saveTree(processPath,dictVar,nCleaned_MX,Y_pred_cut,nameTree="reducedTree_bkg_7")

nCleaned
(268863, 5)
(136840, 5)
(110404, 5)
ttHToGG_M125_13TeV_powheg_pythia8_v2 events with 36 fb-1: 6.34311023972
(44619, 5)
after cutting in dijet mass:2.56716188328


In [17]:
data_count_df = rpd.read_root(utils.IO.dataName[0],"bbggSelectionTree", columns = additionalCut_names)
#w = (np.ones_like(utils.IO.data_df[0].index)).astype(np.int8)
#data_count_df['weight'] = np.multiply(w,data_count_df['isSignal'])

nTot,dictVar = utils.postprocessing.stackFeatures(data_count_df,additionalCut_names,isData=1)


#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,dictVar['weight']]!=0),:][0]
print "nCleaned"
print nCleaned.shape
print Y_pred_data.shape


blind = True
if not blind:
    print Y_pred_data.shape
    nCleanedaned_MVA = nCleaned[np.where(Y_pred_data>MVA_Cut)]
    #apply massWindow +- 1.5
    
    nCleaned_massWindow = utils.postprocessing.cutInvariantMass(nCleaned_MVA,dictVar['diphotonCandidate.M()'],123.5000,126.5000)
    print nCleaned_massWindow.shape
    nEvts = sum(nCleaned_massWindow[:,1])
    print utils.IO.dataName[0].split("/")[len(utils.IO.dataName[0].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
    #Histo_B = np.histogram(nCleaned_massWindow[:,2],bins=30,range=(100,180),weights=nCleaned_massWindow[:,1])
    #import matplotlib.pyplot as plt
    #plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
    #plt.show()
    nCleaned_mdijet = utils.postprocessing.cutInvariantMass(nCleaned_massWindow,dictVar['dijetCandidate.M()'],100.,140.)

    print nCleaned_mdijet.shape
    nEvts_Cut = sum(nCleaned_mdijet[:,dictVar['weight']])
    print "after cutting in dijet mass:"+str(nEvts_Cut)
    processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.dataName[0].split("/")[len(utils.IO.dataName[0].split("/"))-1].replace("output_","")
    utils.postprocessing.saveTree(processPath,dictVar,nCleaned_mdijet)
    
    processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.dataName[0].split("/")[len(utils.IO.dataName[0].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
    utils.postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_data)
    
else:
    
    #save preselection data
    processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.dataName[0].split("/")[len(utils.IO.dataName[0].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
    utils.postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_data)
    
    #do blinding
    nCleaned_MVA = nCleaned[np.where(Y_pred_data>MVA_Cut)]
    
    nCleaned_massWindow = nCleaned_MVA[np.where(np.logical_or(nCleaned_MVA[:,dictVar['diphotonCandidate.M()']]<115,nCleaned_MVA[:,dictVar['diphotonCandidate.M()']]>135))]
#    nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]>135)]
    nCleaned_mdijet = nCleaned_massWindow[np.where(np.logical_or(nCleaned_massWindow[:,dictVar['dijetCandidate.M()']]<100,nCleaned_massWindow[:,dictVar['dijetCandidate.M()']]>140))]
#    nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]>140)]
    
    
    processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.dataName[0].split("/")[len(utils.IO.dataName[0].split("/"))-1].replace("output_","").replace(".root","")+"_blinded"+".root"
    utils.postprocessing.saveTree(processPath,dictVar,nCleaned_mdijet)
    
    processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+"BkgWS.root"
    utils.postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_data,nameTree="reducedTree_bkg")
    
    processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+"BkgWS_highMass.root"
    Y_pred_total = np.column_stack((nCleaned,Y_pred_data))

    nCleaned_MVA_2 = Y_pred_total[np.where(Y_pred_data>MVA_Cut)]
#    nCleaned_massWindow_2 = nCleaned_MVA_2[np.where(np.logical_or(nCleaned_MVA_2[:,dictVar['diphotonCandidate.M()']]<115,nCleaned_MVA_2[:,dictVar['diphotonCandidate.M()']]>135))]
#    nCleaned_mdijet_2 = utils.postprocessing.cutInvariantMass(nCleaned_massWindow_2,dictVar['dijetCandidate.M()'],100.,140.)
    nCleaned_mdijet_2 = utils.postprocessing.cutInvariantMass(nCleaned_MVA_2,dictVar['dijetCandidate.M()'],100.,140.)
    nCleaned_MX = utils.postprocessing.applyCut(nCleaned_mdijet_2,dictVar['MX'],350)


    Y_pred_cut=nCleaned_MX[:,nCleaned_MX.shape[1]-1].astype(np.float64)


    utils.postprocessing.saveTree(processPath,dictVar,nCleaned_MX,Y_pred_cut,nameTree="reducedTree_bkg")


nCleaned
(92390, 5)
(92390,)
